In [41]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score
from sklearn import metrics

from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles
from sklearn.metrics import adjusted_rand_score

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
import plotly.tools as tls # It's useful to we get some tools of plotly
import warnings # This library will be used to ignore some warnings
from collections import Counter # To do counter of some features
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


german = pd.read_table("german.data", delimiter=' ', header=None)
german.columns = ['Status_Account', 'Duration', 'Credit_History', 'Purpose', 'Credit_Amount',
                     'Saving_Account', 'Present', 'Installment', 'Personal_Status', 'Other', 
                         'Residence', 'Property', 'Age', 'Other_Installment', 'Housing', 'Existing_Credits',
                             'Job', 'Liable_People', 'Tel', 'Foreign_Worker', 'Actual_Score']
germanOrigin = german
target = german.iloc[:,-1]   # Enregistrer la varialbe target
# goodGerman = german[german['Actual_Score']==1]
# badGerman = german[german['Actual_Score']==2]
# german = german.iloc[:,0:-1]; # Enleveer la colonne target
# german = german.drop(columns="Tel") # Enlever la colonne tel


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning:

read_table is deprecated, use read_csv instead.



In [4]:
def replaceModalitites(data, racine, cible):
    return data.replace(to_replace=racine, value=cible)

In [42]:
# STATUS ACCOUNT
germanV2 = replaceModalitites(germanOrigin,["A11","A12","A14"],["A11","A11","A11"])

# PURPOSE
germanV2 = replaceModalitites(germanV2,
                              ['A46', 'A42', 'A40', 'A41', 'A49', 'A45', 'A410','A48'],
                              ['A41', 'A41', 'A41', 'A41', 'A41', 'A41', 'A41','A41'])

#SAVING ACCOUNT
germanV2 = replaceModalitites(germanV2,
                              ['A63','A62'],
                              ['A62', 'A62'])
germanV2 = replaceModalitites(germanV2,
                              ['A61','A64','A65'],
                              ['A61', 'A61', 'A61'])

# PRESENT
germanV2 = replaceModalitites(germanV2,
                              ['A75', 'A73', 'A74', 'A72'],
                              ['A72', 'A72', 'A72', 'A72'])

#PERSONAL STATUS
germanV2 = replaceModalitites(germanV2,
                              ['A93', 'A92', 'A91'],
                              ['A91', 'A91', 'A91'])

# PROPERTY
germanV2 = replaceModalitites(germanV2,
                              ['A122', 'A124', 'A123'],
                              ['A122', 'A122', 'A122'])

# JOB
germanV2 = replaceModalitites(germanV2,
                              ['A173', 'A172'],
                              ['A172', 'A172'])

# INSTALLMENT

In [ ]:
germanV2 = germanOrigin

In [43]:
germanV3 = germanV2.drop(["Personal_Status","Purpose","Job","Housing","Other","Tel", "Actual_Score"], axis=1)

In [ ]:
germanV3 = germanV2.drop(["Tel", "Actual_Score"], axis=1)

In [11]:
def QualiQuantiDecomposition(data):
    data_cat = data.select_dtypes(include=['object'])
    data_num = data._get_numeric_data()
    data_num.drop('Duration', axis=1, inplace= True)

    for col in data_cat:
        onehot = pd.get_dummies(data_cat[col])
        data_cat = data_cat.drop(col,axis = 1)
        data_cat = data_cat.join(onehot)
        
        
    cols = []
    qmn = data_num['Credit_Amount'].quantile(q=0.25)
    qmx = data_num['Credit_Amount'].quantile(q=0.75)
    data_num['Credit_Amount'] = data_num['Credit_Amount'].apply(lambda x : qmn if x < qmn else (qmx if x > qmx else x))
        
    return data_num, data_cat
    

In [44]:
german_num, german_cat = QualiQuantiDecomposition(germanV3)

In [6]:
from sklearn.preprocessing import MinMaxScaler
def ScaleMinMax(data_num):
    scaler = MinMaxScaler()
    data_num_sc = pd.DataFrame(scaler.fit_transform(data_num), columns=data_num.columns)
    return data_num_sc
    

In [7]:
from sklearn.preprocessing import StandardScaler
def ScalerStandard(data_num):
    scaler = StandardScaler()
    data_num_sc = pd.DataFrame(scaler.fit_transform(data_num), columns=data_num.columns)
    return data_num_sc
    

In [16]:
from kmodes.kprototypes import KPrototypes

In [17]:
kp = KPrototypes(n_clusters=2)

In [20]:
germanV3.head()

,Status_Account,Duration,Purpose,Credit_Amount,Saving_Account,Present,Installment,Personal_Status,Other,Residence,Property,Age,Job,Foreign_Worker
0,A11,6,A43,1169,A61,A72,4,A91,A101,4,A121,67,A172,A201
1,A11,48,A43,5951,A61,A72,2,A91,A101,2,A121,22,A172,A201
2,A11,12,A41,2096,A61,A72,2,A91,A101,3,A121,49,A172,A201
3,A11,42,A41,7882,A61,A72,2,A91,A103,4,A122,45,A172,A201
4,A11,24,A41,4870,A61,A72,3,A91,A101,4,A122,53,A172,A201


In [22]:
kp.fit(germanV3, categorical=[0,2,4,5,6,7,8,9,10,12,13])

KPrototypes(cat_dissim=<function matching_dissim at 0x000001C3FEF5D1E0>,
            gamma=1116.981357363962, init='Huang', max_iter=100, n_clusters=2,
            n_init=10, n_jobs=1,
            num_dissim=<function euclidean_dissim at 0x000001C3FEF5D158>,
            random_state=None, verbose=0)

In [23]:
pd.crosstab(target,kp.labels_)

col_0,0,1
Actual_Score,,
1,599,101
2,228,72


### Decomposition


In [45]:
german_num, german_cat = QualiQuantiDecomposition(germanV3)

### MINMAX 

In [46]:
data_num_sc = ScaleMinMax(german_num)

data = [data_num_sc,german_cat]
germanV4 = pd.concat(data,axis=1)

In [48]:
germanV4.head()

,Credit_Amount,Installment,Residence,Age,Existing_Credits,Liable_People,A11,A13,A30,A31,...,A62,A71,A72,A121,A122,A141,A142,A143,A201,A202
0,0.000000,1.000000,1.000000,0.857143,0.333333,0.0,1,0,0,0,...,0,0,1,1,0,0,0,1,1,0
1,1.000000,0.333333,0.333333,0.053571,0.000000,0.0,1,0,0,0,...,0,0,1,1,0,0,0,1,1,0
2,0.280234,0.333333,0.666667,0.535714,0.000000,1.0,1,0,0,0,...,0,0,1,1,0,0,0,1,1,0
3,1.000000,0.333333,1.000000,0.464286,0.000000,1.0,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0
4,1.000000,0.666667,1.000000,0.607143,0.333333,1.0,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0


In [49]:
kp.fit(germanV4, categorical=list(range(5,24)))

KPrototypes(cat_dissim=<function matching_dissim at 0x000001C3FEF5D1E0>,
            gamma=1116.981357363962, init='Huang', max_iter=100, n_clusters=2,
            n_init=10, n_jobs=1,
            num_dissim=<function euclidean_dissim at 0x000001C3FEF5D158>,
            random_state=None, verbose=0)

In [50]:
pd.crosstab(target,kp.labels_)

col_0,0,1
Actual_Score,,
1,407,293
2,200,100
